### ***Importando as bibliotecas***

In [1]:
import torch
import shutil
import gc
from datasets import load_dataset
from unsloth import FastLanguageModel, is_bfloat16_supported
from trl import SFTTrainer
from transformers import TrainingArguments

Unsloth: Your Flash Attention 2 installation seems to be broken?
A possible explanation is you have a new CUDA version which isn't
yet compatible with FA2? Please file a ticket to Unsloth or FA2.
We shall now use Xformers instead, which does not have any performance hits!
We found this negligible impact by benchmarking on 1x A100.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


### ***Inicializando o Foundation Model***

O Foundation Model escolhido para o treinamento será o **unsloth/tinyllama-bnb-4bit** por conta do tamanho e a utilização da memória. O max_seq_length será utilizado se baseando na quantidade de tokens que retornou na etapa 1.

In [2]:
max_seq_length = 256 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

### ***Carregando os arquivos do dataset***

Como aqui será feito o treinamento completo, ou seja, com todas as partes do dataset, é criado uma variavel com a lista de nome destes arquivos.

In [3]:
# Lista com os nomes dos arquivos
arquivos = [f'./LF-Amazon-1.3M/treino/trn_formatado_parte_{num}.json' for num in range(1, 11)]

### ***Fine Tuning***

**Argumentos de treinamento**

Os argumentos que serão utilizados aqui são os argumentos finais que foram testados na etapa 3. Com a seguinte diferença:
- *num_train_epochs*: Será utilizado 1 época por conta do tempo mas, ao menos, ira garantir que o modelo viu todos os dados do dataset.

In [4]:
# Parâmetros para o treinamento
trainingArguments = TrainingArguments(
    per_device_train_batch_size=128,
    gradient_accumulation_steps=2,
    warmup_steps=5,
    num_train_epochs=1,
    learning_rate=3e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=100,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs"
)

**Função de treinamento**

Foi realizado algumas modificações comparado com o da etapa 4, ficando da seguinte forma:

**1.** Inicialização do modelo.
- Valida a parte e inicia o modelo, caso for a primeira parte o treinamento será feito no Foundation Model que seria o TinyLlama, após a primeira parte o treinamento começa a ser realizado no modelo salvo.
- Foi feito desta forma para que, ao iniciar o treinamento de um novo dataset, limpe a memória da GPU e inicie o ultimo modelo treinado.

**2.** Carregamento do dataset.
- Inicia o dataset que foi passado como parâmetro.

**3.** Criação do trainer.
- Aqui neste caso não vai ser realizado a avaliação do modelo, apenas o treinamento.

**4.** Realização do treinamento.

- Inicia o treinamento do modelo com o dataset que foi passado.

**5.** Salvar o modelo.
- Por último é realizado a gravação do modelo em uma pasta, fazendo uma identificação de quantas partes foram treinadas até o momento.

In [5]:
def treinar_e_limpar_memoria(arquivo, parte):
    print(f"Lendo e treinando com o arquivo: {arquivo}")
    
    if parte == 0:
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name = "unsloth/tinyllama-bnb-4bit",
            max_seq_length = max_seq_length,
            dtype = dtype,
            load_in_4bit = load_in_4bit,
        )
        
        model = FastLanguageModel.get_peft_model(
            model,
            r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
            target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                            "gate_proj", "up_proj", "down_proj",],
            lora_alpha = 16,
            lora_dropout = 0, # Supports any, but = 0 is optimized
            bias = "none",    # Supports any, but = "none" is optimized
            # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
            use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
            random_state = 3407,
            use_rslora = False,  # We support rank stabilized LoRA
            loftq_config = None, # And LoftQ
        )
    else:
        torch.cuda.empty_cache()  # Limpar a memória da GPU
        gc.collect()  # Forçar a coleta de lixo
        
        # Carregar o último modelo treinado
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name = f"./model/lora_model_parte_{parte}",
            max_seq_length = max_seq_length,
            dtype = dtype,
            load_in_4bit = load_in_4bit,
        )
    
    # Carregar o dataset a partir do arquivo JSON
    dataset = load_dataset("json", data_files=arquivo, split="train")
    
    # Configurar o treinador para cada parte do dataset
    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=dataset,
        dataset_text_field="text",
        max_seq_length=max_seq_length,
        dataset_num_proc=8,  # Multiprocessamento
        packing=False,  # Se necessário para sequências curtas
        args=trainingArguments
    )
    
    # Treinar o modelo
    trainer.train()
    
    if parte == 9:
        # Caminho da pasta do modelo
        pasta_modelo = f"./model/lora_model_final"
    else:
        # Caminho da pasta do modelo
        pasta_modelo = f"./model/lora_model_parte_{parte + 1}"
    
    # Salvar o melhor modelo treinado automaticamente
    trainer.save_model(pasta_modelo)
    tokenizer.save_pretrained(pasta_modelo)
    
    print(f"Treinamento com {arquivo} concluído.\n")

**Treinamento**

Para cada arquivo que foram recuperados na variável *arquivos* é realizado um treinamento em cima do modelo. Também é passado qual parte que esta sendo treinada para a identificação do modelo.

In [6]:
for parte, arquivo in enumerate(arquivos):
    treinar_e_limpar_memoria(arquivo, parte)
    
    parte += 1
    
print("Treinamento concluído.")

Lendo e treinando com o arquivo: ./LF-Amazon-1.3M/treino/trn_formatado_parte_1.json
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 Ti. Max memory: 11.994 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.8 patched 22 layers with 22 QKV layers, 22 O layers and 22 MLP layers.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 139,041 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 128 | Gradient Accumulation steps = 2
\        /    Total batch size = 256 | Total steps = 543
 "-____-"     Number of trainable parameters = 12,615,680


Step,Training Loss
100,1.725300
200,1.576000
300,1.559700
400,1.552100
500,1.543800


Treinamento com ./LF-Amazon-1.3M/treino/trn_formatado_parte_1.json concluído.

Lendo e treinando com o arquivo: ./LF-Amazon-1.3M/treino/trn_formatado_parte_2.json
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 Ti. Max memory: 11.994 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 139,041 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 128 | Gradient Accumulation steps = 2
\        /    Total batch size = 256 | Total steps = 543
 "-____-"     Number of trainable parameters = 12,615,680


...

Step,Training Loss
100,1.443000
200,1.434300
300,1.440700
400,1.437500
500,1.434800


Treinamento com ./LF-Amazon-1.3M/treino/trn_formatado_parte_9.json concluído.

Lendo e treinando com o arquivo: ./LF-Amazon-1.3M/treino/trn_formatado_parte_10.json
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 Ti. Max memory: 11.994 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 139,040 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 128 | Gradient Accumulation steps = 2
\        /    Total batch size = 256 | Total steps = 543
 "-____-"     Number of trainable parameters = 12,615,680


Step,Training Loss
100,1.433400
200,1.429900
300,1.430800
400,1.427600
500,1.420600


Treinamento com ./LF-Amazon-1.3M/treino/trn_formatado_parte_10.json concluído.

Treinamento concluído.


In [7]:
# Compactar a pasta do modelo em um arquivo zip
pasta_modelo = f"./model"
shutil.make_archive(base_name=pasta_modelo, format='zip', root_dir=pasta_modelo)   
print(f"Modelo {pasta_modelo} compactado com sucesso.")

Modelo ./model compactado com sucesso.
